In [68]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
pd.set_option('display.max_columns', 2000)
%matplotlib inline
dmt = pd.read_csv('../data/final_data/DMT_Data_Merged_10.25.17.csv')

In [69]:
dmt.head()

internalId externalId passive_phone_model  passive_bat_charge_avg  \
0       70052     2w4MVh                 NaN                     NaN   
1       70061     2XdEfU              iPhone                6.835596   
2       70087     2XyVyJ                 NaN                     NaN   
3       70074     34fjYg                 NaN                     NaN   
4       70133     3FGrfL                 NaN                     NaN   

   passive_bat_charge_less_5hr  passive_bat_charge_start_mean  \
0                          NaN                            NaN   
1                     0.708333                      52.611111   
2                          NaN                            NaN   
3                          NaN                            NaN   
4                          NaN                            NaN   

   passive_bat_charge_end_mean  passive_bat_charge_increase_mean  \
0                          NaN                               NaN   
1                    75.104167                         22.493056   
2                          NaN                               NaN   
3                          NaN                               NaN   
4                          NaN                               NaN   

   passive_bat_charge_less_20percent  passive_bat_charge_greater_80percent  \
0                                NaN                                   NaN   
1                           0.159722                                  0.25   
2                                NaN                                   NaN   
3                                NaN                                   NaN   
4                                NaN                                   NaN   

   passive_screen_unlock_num  passive_screen_unlock_less_1min  \
0                        NaN                              NaN   
1                      573.0                          0.47993   
2                        NaN                              NaN   
3                        NaN                              NaN   
4                        NaN                              NaN   

   passive_screen_unlock_avg_per_day  passive_screen_unlock_avg_time_min  \
0                                NaN                                 NaN   
1                          27.130371                            4.941992   
2                                NaN                                 NaN   
3                                NaN                                 NaN   
4                                NaN                                 NaN   

   passive_screen_unlock_time_ratio  passive_call_out  passive_call_in  \
0                               NaN               NaN              NaN   
1                           0.09311              22.0             26.0   
2                               NaN               NaN              NaN   
3                               NaN               NaN              NaN   
4                               NaN               NaN              NaN   

   passive_call_total  passive_call_out_ratio  passive_call_avg_min  \
0                 NaN                     NaN                   NaN   
1                48.0                0.458333              2.859375   
2                 NaN                     NaN                   NaN   
3                 NaN                     NaN                   NaN   
4                 NaN                     NaN                   NaN   

   passive_call_out_wait_durations  SAGE_demographics_v2___ROW_ID  \
0                              NaN                           39.0   
1                         8.636364                           50.0   
2                              NaN                           77.0   
3                              NaN                           74.0   
4                              NaN                          149.0   

   SAGE_demographics_v2___ROW_VERSION       SAGE_demographics_v2___recordId  \
0                                16.0  0843556f-1a8a-49a9-9684-5ddc1c204bc6   
1       

## BART

In [70]:
def col_convert_bart(row):
    '''
    Convert a column that contains a string of a list of strings, into a list of floats.  
    
    Inputs
        row: a string of a list of strings.
        
    Output
        returns a list of floats
    
    '''
    if pd.notnull(row):
        return [float(x) for x in row.strip('[]').split(',')]
    
    else:
        return row

def split(row, segment):
    seg = 0, 5
    
    if segment == 2:
        seg = 5, 10
    elif segment == 3:
        seg = 10, 15
    
    if pd.notnull(row):
        return row[seg[0]:seg[1]]
    else:
        return row

In [71]:
names = [
    'SAGE_bart_v4_baseline_BART0.25', 
    'SAGE_bart_v4_baseline_BART250.00', 
    'SAGE_bart_v4_21_day_assessment_BART0.25', 
    'SAGE_bart_v4_21_day_assessment_BART250.00'
]

stds = [
    '___pumps_standard_deviation',
    '___pumps_standard_deviation_first_third',
    '___pumps_standard_deviation_second_third',
    '___pumps_standard_deviation_last_third'
]

In [72]:
for name in names:
    dmt[name + stds[0]] = dmt[name + '___pumps_per_balloon'].apply(col_convert_bart).apply(np.std)
    dmt[name + stds[1]] = dmt[name + '___pumps_per_balloon'].apply(col_convert_bart).apply(lambda x: x if np.any(pd.isnull(x)) else x[:5]).apply(np.std)
    dmt[name + stds[2]] = dmt[name + '___pumps_per_balloon'].apply(col_convert_bart).apply(lambda x: x if np.any(pd.isnull(x)) else x[5:10]).apply(np.std)
    dmt[name + stds[3]] = dmt[name + '___pumps_per_balloon'].apply(col_convert_bart).apply(lambda x: x if np.any(pd.isnull(x)) else x[10:]).apply(np.std)

In [73]:
dmt.loc[45, 'INQUISIT_bart___script.elapsedtime'] = np.nan

## Delay Discounting

In [74]:
def col_convert_dd(row, ix=None):
    '''
    Convert a column that contains a string of a list of strings, into a list of floats.
    
    Inputs
        row: a string of a list of strings.
        
    Output
        returns the list of floats
    
    '''
    if pd.notnull(row) and ix:
        return [float(x) for x in row.strip('[]').split(',')][ix]
    
    elif pd.notnull(row):
        return [float(x) for x in row.strip('[]').split(',')]
    
    else:
        return row

In [75]:
#Create new column using "last" number in 'choiceArray'.
for col in dmt.columns:
    if 'choiceArray' in col and 'delay' in col:
        col_name = col + '_last'
        dmt[col_name] = dmt[col].apply(lambda x: col_convert_dd(x, -1))

In [76]:
#Create new column for times mean.
for col in dmt.columns:
    if 'delay' in col and col.endswith('times'):
        col_name = col + '_mean'
        dmt[col_name] = dmt[col].apply(lambda x: np.mean(col_convert_dd(x)))

In [77]:
dmt.loc[92, 'INQUISIT_delay_current___script.elapsedtime'] = np.nan

## Save File

In [78]:
dmt.to_csv("Updated_DMT_Final.csv")